# pygohome

## Python, Let's Go Home. Quickly.

*pygohome* is a 100% personal route optimizer in a known environment based on experience.

*You* walk/ride/drive frequently between known locations (home, work, school, shops, family, friends, …) using different routes, but would like to know the optimal route, that should take you the less time possible? *pygohome* uses your recorded GPX tracks to build a route network of *your* world with estimation on how long *you* need to get from A to B using the mean of transport of your choice.

## How it works

1. **Choose your mean of transport.** Bicycle works great, walking should too. Motorized vehicles may be served better by real-time online services.

2. **Track all your trips** using that mean of transport. Start the tracking before you leave, stop the tracking after you arrive. Ride/walk/drive normally, stop at lights, don't speed. OsmAnd works great (list of other apps needed). 1 or 2 seconds tracking interval is ok.

3. **Add POIs (and intersections) as waypoints.** Transfer all GPX files to your computer into a directory (OsmAnd names them `YYYY-MM-DD_HH-MM_DDD.gpx`). Load all of them into some GPX editor (JOSM works great). Create a new GPX file and add all points of interest (home, work, or any place where you started, deliberately paused or ended a trip) with an attribute `name=…`. Then add unnamed waypoints on each intersection where your tracks cross, split, or join. Save this as `pois.gpx`.

4. **Convert all GPX files (tracks and pois.gpx) into CSV using gpsbabel.** If all GPX files are in the `gpx` directory and all CSV files in `csv`, then:

```bash
for x in gpx/20*gpx; gpsbabel -t -i gpx -f $x -o unicsv,grid=utm -F csv/${x:t:r}.csv
    
gpsbabel -w -i gpx -f gpx/pois.gpx -o unicsv,grid=utm -F csv/pois.csv
```

5. **Run the cell below**

6. **Call the method `fastest_path(src, dst, quantile=0.8)`** where `src` and `dst` are the names of the POIs as string and `quantile` is the a float number between `0.0` (take into account your best time on each road segment) and `1.0` (worst time), while `0.8` is a safe value.

In [1]:
from pathlib import Path
import math
import statistics

import networkx as nx
import numpy as np
import ipyleaflet as lf
import pandas as pd
from scipy.spatial import cKDTree
import utm

# load nodes (POIs and intersections)
nodes = pd.read_csv(
    "csv/pois.csv", usecols=["Name", "UTM-Zone", "UTM-Ch", "UTM-East", "UTM-North"]
)
pois = {name: i for i, name in nodes["Name"].items() if not name.startswith("WPT")}

# load all tracks into a single DataFrame
dfr = pd.concat(
    [
        pd.read_csv(
            path,
            usecols=["Date", "Time", "UTM-Zone", "UTM-Ch", "UTM-East", "UTM-North"],
            parse_dates=[["Date", "Time"]],
        )
        for path in sorted(Path("csv").glob("20*.csv"))
    ]
).sort_values("Date_Time")

# make sure all tracks and POIs are in the same UTM zone:
utm_zones = set(nodes["UTM-Zone"]) | set(dfr["UTM-Zone"])
utm_chs = set(nodes["UTM-Ch"]) | set(dfr["UTM-Ch"])
if len(utm_zones) == len(utm_chs) == 1:
    utm_zone = utm_zones.pop()
    utm_ch = utm_chs.pop()
else:
    raise Exception(f"Not a unique UTM Zone ({utm_zones!r}, {utm_chs!r})")

# split into segments (at least 1 minute break between points)
dfr["segment"] = (dfr["Date_Time"].diff() > pd.Timedelta("00:01:00")).cumsum()

# calculate offset to the first record of each segment in seconds
dfr["offset"] = (
    dfr["Date_Time"] - dfr.groupby("segment")["Date_Time"].transform("first")
).dt.total_seconds()

# build a KDTree of all nodes and find if trackpoints are closer than 30 meters from them
tree = cKDTree(nodes[["UTM-East", "UTM-North"]])
dfr["node"] = tree.query(dfr[["UTM-East", "UTM-North"]], distance_upper_bound=30)[1]
dfr = dfr[dfr.node < tree.n]

# for each passage by a node get the offset of the first and the last point
dfr = dfr.groupby((dfr.node != dfr.groupby("segment")["node"].shift(1)).cumsum()).agg(
    segment=("segment", "first"),
    start=("offset", "first"),
    end=("offset", "last"),
    node=("node", "first"),
)

# `pred_secs` between leaving 30m radius of `pred_node` and entering 30m radius of `curr_node`
# `curr_secs` between entering and leaving the 30m radius of `curr_node`
# `succ_secs` between leaving 30m radius of `curr_node` and entering 30m radius of `succ_node`
pred_dfr = dfr.groupby("segment").shift(1, fill_value=-1)
succ_dfr = dfr.groupby("segment").shift(-1, fill_value=-1)
df2 = pd.DataFrame(
    {
        "pred_node": pred_dfr["node"],
        "curr_node": dfr["node"],
        "succ_node": succ_dfr["node"],
        "pred_secs": dfr["start"] - pred_dfr["end"],
        "curr_secs": dfr["end"] - dfr["start"],
        "succ_secs": succ_dfr["start"] - dfr["end"],
    }
)

# gpsbabel names unnamed waypoints as WPTxxxxx
is_poi = ~df2.join(nodes, on="curr_node")["Name"].str.startswith("WPT")

# an intersection is "slow" (traffic lights)
# if at least 25% of tracks spend more than 20 seconds within 30 meters of its node
is_slow = df2.groupby("curr_node")["curr_secs"].transform(
    lambda x: x.quantile(0.75) > 20
)

# build the graph
g = nx.DiGraph()
grp_slow = (
    df2[~is_poi & is_slow]
    .query("pred_node >=0 and succ_node >= 0")
    .groupby(["pred_node", "curr_node", "succ_node"])
)
g.add_edges_from(
    (
        ((c, p, c), (c, c, s), {"secs": sorted(grp["curr_secs"])})
        for (p, c, s), grp in grp_slow
    )
)

df2["succ_secs"] += df2[~is_poi & ~is_slow]["curr_secs"].reindex(df2.index).fillna(0)
df2.loc[~is_poi & ~is_slow, "curr_secs"] = 0

grp_simple = df2.query("succ_node >= 0").groupby(["curr_node", "succ_node"])
g.add_edges_from(
    (
        (
            (c, c, s) if (c, c, s) in g else c,
            (s, c, s) if (s, c, s) in g else s,
            {"secs": sorted(grp["succ_secs"])},
        )
        for (c, s), grp in grp_simple
    )
)

pos = {i: np.array([rec["UTM-East"], rec["UTM-North"]]) for i, rec in nodes.iterrows()}
for node in g.nodes:
    if isinstance(node, tuple):
        # nodes in slow intersections are positioned on a circle
        # of radius 30m in the direction of the predecessor/successor
        # and always on the right side
        # (has no impact on results, only on the map)
        here, src, dst = node
        step = (30 * (pos[dst] - pos[src]) / math.hypot(*(pos[dst] - pos[src]))).astype(
            int
        )
        if here == src:
            pos[node] = pos[here] + step + np.array([step[1], -step[0]]) // 6
        else:
            pos[node] = pos[here] - step - np.array([step[1], -step[0]]) // 6


def fastest_path(src, dst, quantile=0.8):
    src_poi = pois[src]
    dst_poi = pois[dst]

    src_latlon = utm.to_latlon(pos[src_poi][0], pos[src_poi][1], utm_zone, utm_ch)
    m = lf.Map(
        center=src_latlon,
        zoom=14,
        interpolation="nearest",
        basemap=lf.basemaps.CartoDB.DarkMatter,
    )

    sp = nx.path_graph(
        nx.dijkstra_path(
            g, src_poi, dst_poi, lambda u, v, a: np.quantile(a["secs"], quantile)
        )
    )
    line = lf.AntPath(
        locations=[
            utm.to_latlon(pos[node][0], pos[node][1], utm_zone, utm_ch) for node in sp
        ]
    )

    m.add_layer(line)
    nd = sum(
        statistics.NormalDist.from_samples(g.edges[edge]["secs"])
        if len(g.edges[edge]["secs"]) > 1
        else statistics.NormalDist(g.edges[edge]["secs"][0], 0)
        for edge in sp.edges
    )
    qu = sum(np.quantile(g.edges[edge]["secs"], quantile) for edge in sp.edges)

    print(
        "quantile {:.2f}: {:2d}:{:02d} (norm: {:2d}:{:02d}±{:d}:{:02d})   "
        "{} (nb of measurements/segment)".format(
            quantile,
            *divmod(int(qu), 60),
            *divmod(int(nd.mean), 60),
            *divmod(int(nd.stdev), 60),
            "·".join(str(len(g.edges[edge]["secs"])) for edge in sp.edges),
        )
    )
    return m


In [ ]:
fastest_path("hbf", "schloss", 0.8)